# California housing prices 
.
Versão com o PySpark do kernel [California housing prices](https://github.com/robson-rsp/datascience/blob/main/regression/california_housing_prices.ipynb). Este notebook foi escrito no ambiente do Databricks e exportado como arquivo IPython .ipynb. Os textos daqui são cópias dos originais do link.
---
A tarefa neste kernel é criar um modelo de Machine Learning que estime preços de imóveis no estado da Califórnia baseado em características, como:

*   latitude/longitude: Localização do imóvel
*   housing_median_age: Idade média dos imóveis naquela quadra
*   totalRooms: Número total de cômodos dentro de um quarteirão
*   totalBedrooms: Número total de quartos dentro de um quarteirão
*   population: Número total de pessoas que moram em uma quadra
*   households: Número total de habitações em uma quadra
*   medianIncome: Renda média das famílias de uma quadra (medida em dezenas de milhares de dólares americanos)
*   medianHouseValue: Valor médio dos imóveis de uma quadra (medido em dólares americanos)
*   oceanProximity: Localização do imóvel em relação ao oceano/mar

Obs. A média populacional desses quarteirões é de 600 até 3000 pessoas.

Fonte: [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

In [0]:

housing = spark.read.format("csv") \
          .option("inferSchema", "True") \
          .option("header", "True") \
          .option("sep", ",") \
          .load("/FileStore/tables/housing.csv")

# EDA

In [0]:
housing.sample(fraction=0.01).toPandas()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.27,37.85,42.0,1639.0,367.0,929.0,366.0,1.7135,159800.0,NEAR BAY
1,-122.24,37.80,52.0,996.0,228.0,731.0,228.0,2.2697,127000.0,NEAR BAY
2,-122.23,37.78,44.0,2340.0,825.0,2813.0,751.0,1.6009,118100.0,NEAR BAY
3,-122.17,37.76,41.0,1594.0,367.0,1074.0,355.0,1.9356,90600.0,NEAR BAY
4,-122.28,37.87,49.0,2026.0,548.0,963.0,521.0,1.9805,173700.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
201,-119.17,34.18,38.0,3221.0,783.0,2792.0,736.0,2.9118,172400.0,NEAR OCEAN
202,-119.15,34.25,36.0,3511.0,664.0,2965.0,695.0,4.0878,186800.0,NEAR OCEAN
203,-118.83,34.15,16.0,3380.0,731.0,1227.0,641.0,4.2857,233200.0,NEAR OCEAN
204,-118.73,34.29,8.0,4983.0,754.0,2510.0,725.0,6.9454,276500.0,<1H OCEAN


In [0]:
print(f'shape={housing.count(), len(housing.columns)}')

shape=(20640, 10)


In [0]:
housing.summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max").toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,25%,-121.8,33.93,18.0,1447.0,296.0,787.0,280.0,2.5625,119600.0,None
5,50%,-118.49,34.26,29.0,2127.0,435.0,1166.0,409.0,3.5347,179700.0,None
6,75%,-118.01,37.71,37.0,3146.0,647.0,1724.0,605.0,4.7426,264700.0,None
7,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


Verificando a ocorrência de campos NaN.

In [0]:
for name in housing.columns:
    mask = housing[name].isNull()
    print(f'{name}: {housing.where(mask).count()}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


# Data visualization

Os gráficos foram criados utilizando as ferramentas de visualização nativas do Databricks. O resultado não pode ser visto em um arquivo .ipynb

# Train/Test split

In [0]:
train_set, test_set = housing.randomSplit([0.8, 0.2])

# Feature engineering

In [0]:
from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler

## Categorical features

**Etapa ##:** Atributos nominais: Codificar

In [0]:
si = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_')
si_model = si.fit(train_set)
train_set = si_model.transform(train_set)
test_set  = si_model.transform(test_set)

## Numerical features

**Etapa ##:** NaN values: Imputar

In [0]:
im = Imputer(strategy='mean', inputCol='total_bedrooms', outputCol='total_bedrooms_')
im_model = im.fit(train_set)
train_set = im_model.transform(train_set)
test_set  = im_model.transform(test_set)